# Aquisição e Manipulação de Dados

## Modelagem do Futebol Brasileiro

### Autores: Hanna Rodrigues e Igor Michels

_________________________

### Import's

In [1]:
from functions import *
from random import choice
from glob import glob
import numpy as np
import matplotlib
from datetime import timedelta

_________________________

### Objetivos do negócio:

O projeto objetiva a criação de um modelo de predição de resultados gerais dos jogos do futebol brasileiro em vitória, derrota ou empate, bem como a inferência do campeão brasileiro utilizando um modelo que será implementado utilizando dados disponíveis sobre jogos da Série A do Brasileirão que podem ser encontrados no site da CBF e em outros sites esportivos.


### Avaliação da situação atual:

Atualmente casas de apostas possuem bons modelos preditivos para a elaboração das odds, enquanto modelos mais simples são utilizados, seja em pesquisa ou seja com objetivos didáticos. Além disso, existem muitos dados disponíveis acerca do tema, uma vez que o futebol é o esporte mais popular do planeta, ou seja, pode-se encontrar os mais variados dados facilmente.


### Objetivo da mineração de dados:

A coleta de dados objetiva, inicialmente, os dados mais gerais de uma partida:

 + mandante;
 + visitante;
 + placar e;
 + data.

Tais dados podem ser obtidos diretamente do site da CBF, como realizado [nesse projeto](https://github.com/IgorMichels/Brazilian_Soccer).


### Plano do projeto:
    
 + Coletar dados do futebol brasileiro via scraping
     + ver [esse repositório](https://github.com/IgorMichels/Brazilian_Soccer/tree/main/Scrape_and_some_views/All%20data/Serie%20A)
 + Explorar e verificar a qualidade dos dados coletados
 + Selecionar, limpar e reformatar os dados que alimentarão o modelo
 + Construir e treinar o modelo de previsão de partidas
     + elaborar modelo estimando forças para cada clube, utilizando os jogos anteriores para realizar o treinamento, com jogos mais recentes tendo peso maior
 + Validar e analisar as previsões do modelo
     + para essa etapa a ideia é utilizar os dados do começo dos campeonatos de 2019 e 2020 (10 primeiras rodadas ou primeiro turno inteiro) para realizar o treinamento do modelo e utilizar os parâmetros obtidos para validar o modelo nesses anos
 + Inferir o campeão brasileiro com o modelo implementado
 + Revisar e fazer o deploy do projeto


## II. Data Understanding  (apagar célula)

+ Collect initial data:  
+ Describe data:  
+ Explore data:  
+ Verify data quality:  

# Entendimento dos Dados

### Coleta Inicial dos Dados

A coleta inicial dos dados já foi realizada em um projeto anterior, conforme seção acima. Dessa forma, realizamos apenas o download dos dados para esse projeto, considerando todos os jogos de todas as competições da CBF no período de 2013 a 2020.

### Descrição dos Dados

Os dados coletados estão no formato ```csv```, com a súmula crua e completa de cada partida, conforme pode ser visto abaixo.

In [2]:
game = 'Data/Serie A/2020/Game 001.csv' # Flamengo e Atlético-MG pela primeira rodada de 2020
lines_2_print = 25
lines = 0
with open(game) as file:
    reader = csv.reader(file, delimiter = ',')
    for row in reader:
        print(row)
        
        lines += 1
        if lines == lines_2_print:
            break

['Campeonato:', 'Campeonato Brasileiro - Série A / 2020', 'Rodada:', '1', '', '']
['Jogo:', 'Flamengo / RJ X Atlético / MG', '', '', '', '']
['Data:', '09/08/2020', 'Horário:', '16:00', 'Estádio:', 'Jornalista Mário Filho / Rio de Janeiro']
['Arbitragem', '']
['Arbitro:', 'Raphael Claus (FIFA / SP)']
['Arbitro Assistente 1:', 'Daniel Paulo Ziolli (AB / SP)']
['Arbitro Assistente 2:', 'Anderson José de Moraes Coelho (AB / SP)']
['Quarto Arbitro:', 'Rafael Martins de Sá (CD / RJ)']
['Analista de Campo:', 'Edilson Soares da Silva (CBF / RJ)']
['VAR:', 'Thiago Duarte Peixoto (AB / SP)']
['AVAR1:', 'Vinicius Furlan (AB / SP)']
['AVAR2:', 'Enderson Emanoel Turbiani da Silva (AB / SP)']
['Observador de VAR:', 'Nilson de Souza Monção (CBF / RJ)']
['Quality manager:', 'Mikael Silva de Araujo (CBF / RJ)']
['Cronologia', '', '', '', '', '', '', '']
['1o Tempo', '2o Tempo', '', '', '', '', '', '']
['Entrada do mandante:', '15:50', 'Atraso:', 'Não Houve', 'Entrada do mandante:', '16:57', 'Atraso:',

Os dados da súmula foram retirados de um ```PDF``` e convertidos para ```csv``` por meio da biblioteca ```tabula```.

### Exploração dos Dados

Essa etapa pensamos em adiar um pouco para realizar após a limpeza dos dados, uma vez que os mesmos precisam ser tratados. Mas uma análise rápida dos dados nos mostra que todos os dados de nosso interesse se encontram no ínicio da súmula, antes da relação de jogadores.

Podemos ver que até esse ponto já pudemos coletar informações como os clubes que estão se enfrentando, a data da partida e o resultado final do jogo:

In [3]:
with open(game) as file:
    reader = csv.reader(file, delimiter = ',')
    for row in reader:
        if 'Relação de Jogadores' in row:
            break
            
        print(row)

['Campeonato:', 'Campeonato Brasileiro - Série A / 2020', 'Rodada:', '1', '', '']
['Jogo:', 'Flamengo / RJ X Atlético / MG', '', '', '', '']
['Data:', '09/08/2020', 'Horário:', '16:00', 'Estádio:', 'Jornalista Mário Filho / Rio de Janeiro']
['Arbitragem', '']
['Arbitro:', 'Raphael Claus (FIFA / SP)']
['Arbitro Assistente 1:', 'Daniel Paulo Ziolli (AB / SP)']
['Arbitro Assistente 2:', 'Anderson José de Moraes Coelho (AB / SP)']
['Quarto Arbitro:', 'Rafael Martins de Sá (CD / RJ)']
['Analista de Campo:', 'Edilson Soares da Silva (CBF / RJ)']
['VAR:', 'Thiago Duarte Peixoto (AB / SP)']
['AVAR1:', 'Vinicius Furlan (AB / SP)']
['AVAR2:', 'Enderson Emanoel Turbiani da Silva (AB / SP)']
['Observador de VAR:', 'Nilson de Souza Monção (CBF / RJ)']
['Quality manager:', 'Mikael Silva de Araujo (CBF / RJ)']
['Cronologia', '', '', '', '', '', '', '']
['1o Tempo', '2o Tempo', '', '', '', '', '', '']
['Entrada do mandante:', '15:50', 'Atraso:', 'Não Houve', 'Entrada do mandante:', '16:57', 'Atraso:',

### Verificando a Qualidade dos Dados

Essa etapa acaba sendo rápida nesse caso, pois os dados foram obtidos das súmulas das partidas que estão no site da CBF, ou seja, são dados oficiais de cada jogo e que se encontram no site oficial do organizador do evento.

## III. Data Preparation (apagar célula)

+ Select data:   
+ Clean data:   
+ Construct data/transform data/feature engineering:   
+ Integrate data:   
+ (Re) Format data:   

# Preparação dos Dados

### Seleção e Limpeza dos Dados

Conforme citado anteriormente, tomamos os dados de todos os jogos de todas as competições da CBF de 2013 a 2020. Como estamos interessados na modelagem da primeira divisão, nessa etapa iremos fazer a seleção dos jogos entre os clubes que já estiveram na Série A, em qualquer que seja a competição.

Pensando em otimizar o processo, decidimos unificar a etapa de seleção e limpeza dos dados. Dessa forma, enquanto passamos por cada jogo, já iremos ver se os clubes que estão se enfrentando estiveram na Série A em algum desses anos. Se sim, então a gente guarda os dados dessa partida, caso contrário essa partida não será de grande valia para nossa análise, então descartamos a mesma.

**OBS:** A ideia de pegar todas as competições se dá pelo fato de que, na Série A, ocorre, a cada ano, a renovação de 20% dos clubes (4 são rebaixados e 4 são promovidos), então para termos dados prévios desses clubes que foram promovidos uma opção é a de utilizar os jogos de divisões inferiores.

In [4]:
def cleaning_data(competitions1 = None, competitions2 = None, output_file = None):
    if competitions1 == None and competitions2 == None:
        competitions = sorted(glob('Data/S*')) + glob('Data/C*') # garantir que a Série A seja a primeira da lista
        competitions1 = [competitions[0]] # only Serie A
        competitions2 = competitions[1:]
    
    if output_file == None:
        output_file = 'data.csv'
    
    save = []
    clubs = []
    for competition in competitions1:
        years = sorted(glob(competition + '/*'))
        for year in years:
            games = sorted(glob(year + '/Game *'))
            for game in games:
                save.append([game[5:game.find('/', 5)]])
                try:
                    teams = find_teams(game)
                    date = find_date(game)
                    score = find_score(game)
                    for team in teams:
                        save[-1].append(team)
                        if team not in clubs:
                            clubs.append(team)

                    save[-1].append(score)
                    for info in date:
                        save[-1].append(info)

                except UnicodeDecodeError:
                    print('UnicodeDecodeError at ' + game)
                    save.pop()

    if competitions2 != None:
        for competition in competitions2:
            years = sorted(glob(competition + '/*'))
            for year in years:
                games = sorted(glob(year + '/Game *'))
                for game in games:
                    save.append([game[5:game.find('/', 5)]])
                    try:
                        teams = find_teams(game)
                        if len(teams) == 2 and teams[0] in clubs and teams[1] in clubs:
                            date = find_date(game)
                            score = find_score(game)
                            for team in teams:
                                save[-1].append(team)

                            save[-1].append(score)
                            for info in date:
                                save[-1].append(info)
                        else:
                            save.pop()

                    except UnicodeDecodeError:
                        # CdB - 2016 - game 122 is Fluminense vs Ypiranga
                        print('UnicodeDecodeError at ' + game)
                        save.pop()

    with open(output_file, 'w') as file:
        writer = csv.writer(file)
        writer.writerow(['Competition', 'Team 1', 'Team 2', 'Result', 'Round', 'Year', 'Date'])
        for game in save:
            if len(game[3]) > 3 and game[3][2] == 'x':
                writer.writerow(game[:7])
            
def calculate_missing_date(df, interval, column):
    beg, end = interval
    years = {'all' : [0 , 0]}
    for i in range(beg, end):
        years['all'][1] += 1
        if type(df.loc[i, column]) != str:
            # não temos a data
            years['all'][0] += 1
            if df.loc[i, 'Year'] not in years:
                years[df.loc[i, 'Year']] = [1, 1]
            else:
                years[df.loc[i, 'Year']][0] += 1
                years[df.loc[i, 'Year']][1] += 1
        else:
            # temos a data
            if df.loc[i, 'Year'] not in years:
                years[df.loc[i, 'Year']] = [0, 1]
            else:
                years[df.loc[i, 'Year']][1] += 1

    return years, years['all'][0]/years['all'][1]
# cleaning_data()

Tendo realizado essa etapa, obtemos um arquivo ```csv``` contendo apenas os dados dos jogos de nosso interesse e, agora sim, vamos a Exploração dos Dados.

**OBS:** a última linha está comentanda pois já realizamos esse processo uma vez, então não é necessário fazer isso novamente cada vez que abrimos o notebook.

### Exploração dos Dados

Uma ideia inicial para essa etapa é visualizar nossos dados em um ```dataframe```, possibilitando uma visão geral dos mesmos.

In [5]:
df = pd.read_csv('data.csv')
df

,Competition,Team 1,Team 2,Result,Round,Year,Date
0,Serie A,Vasco da Gama / RJ,Portuguesa / SP,1 x 0,1,2013,25/05/2013
1,Serie A,Fluminense / RJ,Athlético / PR,2 x 1,1,2013,NaN
2,Serie A,Corinthians / SP,Botafogo / RJ,1 x 1,1,2013,NaN
3,Serie A,Vitória / BA,Internacional / RS,2 x 2,1,2013,NaN
4,Serie A,Grêmio / RS,Náutico / PE,2 x 0,1,2013,NaN
...,...,...,...,...,...,...,...
4446,Copa do Brasil,Bahia / BA,Atlético / MG,2 x 1,11,2021,04/08/2021
4447,Copa do Brasil,Athlético / PR,Santos / SP,1 x 0,11,2021,25/08/2021
4448,Copa do Brasil,Grêmio / RS,Flamengo / RJ,0 x 4,11,2021,25/08/2021
4449,Copa do Brasil,São Paulo / SP,Fortaleza / CE,2 x 2,12,2021,25/08/2021


Podemos ver que existem muitos dados faltantes na coluna Date. Isso nos dá duas opções:

+ excluir essa coluna e considerar as colunas Year e Round como lapso temporal (isso implicaria na exclusão das partidas da Copa do Brasil, que, como veremos abaixo, são 380 ao todo);
+ ou usar os valores que temos para inferir os valores faltantes (os jogos de uma mesma rodada tendem a ser no mesmo intervalo de 2 a 3 dias, o que facilitaria o processo).

Vale ressaltar que os jogos, mesmo dentro de uma mesma rodada, não estão em ordem cronológica.

In [6]:
count = 0
for i in range(4451):
    if df.loc[i, 'Competition'] == 'Copa do Brasil':
        count += 1
        
count

380

Nossa opção se dará pelo segundo caminho, ou seja, buscar inferir os valores faltantes por meio dos valores existentes. Para isso, vamos usar a interpolação pelos valores próximos. O código nas linhas 7 a 15 são para corrigir um problema de data, onde alguns valores estava sendo considerados no formato mm/dd/aaaa.

In [7]:
df['Date_by_Nearest'] = matplotlib.dates.date2num(pd.to_datetime(df['Date']))
df['Date_by_Nearest'].interpolate(method = 'nearest', inplace = True)
df['Date_by_Nearest'] = df['Date_by_Nearest'].apply(np.round)
df['Date_by_Nearest'] = matplotlib.dates.num2date(df['Date_by_Nearest'])
df['Date_by_Nearest'] = df['Date_by_Nearest'].dt.strftime('%d/%m/%Y')

nearest_date = list(df['Date_by_Nearest'])
for i in range(len(nearest_date)):
    if int(nearest_date[i][3:5]) > 12:
        nearest_date[i] = nearest_date[i][3:5] + '/' + nearest_date[i][:2] + '/' + nearest_date[i][-4:]
    else:
        if int(nearest_date[i][:2]) < 12:
            nearest_date[i] = nearest_date[i][3:5] + '/' + nearest_date[i][:2] + '/' + nearest_date[i][-4:]
            
df['Date_by_Nearest'] = nearest_date
df

,Competition,Team 1,Team 2,Result,Round,Year,Date,Date_by_Nearest
0,Serie A,Vasco da Gama / RJ,Portuguesa / SP,1 x 0,1,2013,25/05/2013,25/05/2013
1,Serie A,Fluminense / RJ,Athlético / PR,2 x 1,1,2013,NaN,25/05/2013
2,Serie A,Corinthians / SP,Botafogo / RJ,1 x 1,1,2013,NaN,25/05/2013
3,Serie A,Vitória / BA,Internacional / RS,2 x 2,1,2013,NaN,25/05/2013
4,Serie A,Grêmio / RS,Náutico / PE,2 x 0,1,2013,NaN,25/05/2013
...,...,...,...,...,...,...,...,...
4446,Copa do Brasil,Bahia / BA,Atlético / MG,2 x 1,11,2021,04/08/2021,04/08/2021
4447,Copa do Brasil,Athlético / PR,Santos / SP,1 x 0,11,2021,25/08/2021,25/08/2021
4448,Copa do Brasil,Grêmio / RS,Flamengo / RJ,0 x 4,11,2021,25/08/2021,25/08/2021
4449,Copa do Brasil,São Paulo / SP,Fortaleza / CE,2 x 2,12,2021,25/08/2021,25/08/2021


Mas note que isso gera algumas coisas estranhas, como a Série A de 2016, em que os jogos estão variando de junho de 2015 a maio de 2017 quando, na verdade, o campeonato ocorreu no período de [14 de maio de 2016 a 11 de dezembro de 2016](https://pt.wikipedia.org/wiki/Campeonato_Brasileiro_de_Futebol_de_2016_-_S%C3%A9rie_A):

In [8]:
df.loc[1140:1519]

,Competition,Team 1,Team 2,Result,Round,Year,Date,Date_by_Nearest
1140,Serie A,Palmeiras / SP,Athlético / PR,4 x 0,1,2016,NaN,12/06/2015
1141,Serie A,Flamengo / RJ,Sport / PE,1 x 0,1,2016,NaN,12/06/2015
1142,Serie A,Atlético / MG,Santos / SP,1 x 0,1,2016,NaN,12/06/2015
1143,Serie A,Coritiba / PR,Cruzeiro / MG,1 x 0,1,2016,NaN,12/06/2015
1144,Serie A,Botafogo / RJ,São Paulo / SP,0 x 1,1,2016,NaN,12/06/2015
...,...,...,...,...,...,...,...,...
1515,Serie A,Grêmio / RS,Botafogo / RJ,0 x 1,38,2016,NaN,14/05/2017
1516,Serie A,Athlético / PR,Flamengo / RJ,0 x 0,38,2016,NaN,14/05/2017
1517,Serie A,Chapecoense / SC,Atlético / MG,0 x 0,38,2016,NaN,14/05/2017
1518,Serie A,Ponte Preta / SP,Coritiba / PR,2 x 0,38,2016,NaN,14/05/2017


Para tentar driblar isso, vamos tentar uma abordagem um pouco diferente, considerando as datas existentes em uma rodada do ano desejado, quando disponíveis em grande quantidade, como, por exemplo, nas rodadas 34 e 35 de 2015:

In [9]:
df.loc[1090:1109]

,Competition,Team 1,Team 2,Result,Round,Year,Date,Date_by_Nearest
1090,Serie A,Internacional / RS,Ponte Preta / SP,1 x 0,34,2015,07/11/2015,07/11/2015
1091,Serie A,Corinthians / SP,Coritiba / PR,2 x 1,34,2015,07/11/2015,07/11/2015
1092,Serie A,Athlético / PR,Avaí / SC,2 x 1,34,2015,07/11/2015,07/11/2015
1093,Serie A,Fluminense / RJ,Chapecoense / SC,2 x 3,34,2015,07/11/2015,07/11/2015
1094,Serie A,Palmeiras / SP,Vasco da Gama / RJ,0 x 2,34,2015,08/11/2015,08/11/2015
1095,Serie A,Cruzeiro / MG,São Paulo / SP,2 x 1,34,2015,08/11/2015,08/11/2015
1096,Serie A,Figueirense / SC,Atlético / MG,0 x 1,34,2015,NaN,08/11/2015
1097,Serie A,Flamengo / RJ,Goiás / GO,4 x 1,34,2015,NaN,08/11/2015
1098,Serie A,Sport / PE,Grêmio / RS,1 x 0,34,2015,08/11/2015,08/11/2015
1099,Serie A,Joinville / SC,Santos / SP,0 x 0,34,2015,08/11/2015,08/11/2015


Note que duas datas bem prováveis para os jogos 1096 e 1097 são os dias 07/11/2015 e 08/11/2015 (que foi o resultado da interpolação por proximidade). O mesmo ocorre para os jogos da rodada 35 (jogos 1101, 1103, 1107, 1108).

Por outro lado, as rodadas 1 e 2 de 2016, por exemplo, não podem ser inferidas por datas próximas, uma vez que nenhum dos jogos tem data:

In [10]:
df.loc[1140:1149] # primeira rodada de 2016

,Competition,Team 1,Team 2,Result,Round,Year,Date,Date_by_Nearest
1140,Serie A,Palmeiras / SP,Athlético / PR,4 x 0,1,2016,NaN,12/06/2015
1141,Serie A,Flamengo / RJ,Sport / PE,1 x 0,1,2016,NaN,12/06/2015
1142,Serie A,Atlético / MG,Santos / SP,1 x 0,1,2016,NaN,12/06/2015
1143,Serie A,Coritiba / PR,Cruzeiro / MG,1 x 0,1,2016,NaN,12/06/2015
1144,Serie A,Botafogo / RJ,São Paulo / SP,0 x 1,1,2016,NaN,12/06/2015
1145,Serie A,Santa Cruz / PE,Vitória / BA,4 x 1,1,2016,NaN,12/06/2015
1146,Serie A,Corinthians / SP,Grêmio / RS,0 x 0,1,2016,NaN,12/06/2015
1147,Serie A,Figueirense / SC,Ponte Preta / SP,0 x 0,1,2016,NaN,12/06/2015
1148,Serie A,América / MG,Fluminense / RJ,0 x 1,1,2016,NaN,12/06/2015
1149,Serie A,Internacional / RS,Chapecoense / SC,0 x 0,1,2016,NaN,12/06/2015


Nesses casos, vamos considerar as datas das respectivas rodadas em outros anos, isso é, os jogos da primeira rodada de 2017 foram realizados entre 13 e 15 de maio, logo, podemos pensar que a primeira rodada do campeonato de 2016 tenha sido realizada aproximadamente nessas datas também. Entretanto, tomar isso como verdade pode acarretar erros, então é interessante analisar as rodadas de diversos anos, tomando uma espécie de média.

In [11]:
df.loc[1520:1529] # primeira rodada de 2017

,Competition,Team 1,Team 2,Result,Round,Year,Date,Date_by_Nearest
1520,Serie A,Fluminense / RJ,Santos / SP,3 x 2,1,2017,14/05/2017,14/05/2017
1521,Serie A,Flamengo / RJ,Atlético / MG,1 x 1,1,2017,13/05/2017,13/05/2017
1522,Serie A,Palmeiras / SP,Vasco da Gama / RJ,4 x 0,1,2017,14/05/2017,14/05/2017
1523,Serie A,Corinthians / SP,Chapecoense / SC,1 x 1,1,2017,13/05/2017,13/05/2017
1524,Serie A,Cruzeiro / MG,São Paulo / SP,1 x 0,1,2017,14/05/2017,14/05/2017
1525,Serie A,Coritiba / PR,Atlético / GO,4 x 1,1,2017,15/05/2017,15/05/2017
1526,Serie A,Grêmio / RS,Botafogo / RJ,2 x 0,1,2017,14/05/2017,14/05/2017
1527,Serie A,Bahia / BA,Athlético / PR,6 x 2,1,2017,14/05/2017,14/05/2017
1528,Serie A,Ponte Preta / SP,Sport / PE,4 x 0,1,2017,14/05/2017,14/05/2017
1529,Serie A,Avaí / SC,Vitória / BA,0 x 0,1,2017,14/05/2017,14/05/2017


Vamos, então, ver qunatos jogos da Série A estão sem data em cada ano e em cada rodada.

In [12]:
fix = []
count = {}
for i in range(3221):
    if type(df.loc[i, 'Date']) != str:
        year = df.loc[i, 'Year']
        rd = df.loc[i, 'Round']
        fix.append([i, year, rd])
        if year in count:
            if rd in count[year]:
                count[year][rd] += 1
            else:
                count[year][rd] = 1
        
        else:
            count[year] = {}
            count[year][rd] = 1

print('Jogos sem data:', len(fix))
count

Jogos sem data: 923


{2013: {1: 8,
  2: 7,
  3: 9,
  4: 9,
  5: 10,
  6: 10,
  7: 7,
  8: 9,
  9: 10,
  10: 10,
  11: 10,
  12: 10,
  13: 9,
  14: 10,
  15: 9,
  16: 10,
  17: 9,
  18: 10,
  19: 9,
  20: 10,
  21: 10,
  22: 10,
  23: 8,
  24: 10,
  25: 10,
  26: 9},
 2014: {38: 3},
 2015: {1: 10,
  2: 10,
  3: 10,
  4: 10,
  5: 10,
  6: 10,
  7: 10,
  8: 10,
  9: 10,
  10: 10,
  11: 10,
  12: 10,
  13: 10,
  14: 10,
  15: 10,
  16: 10,
  17: 10,
  18: 10,
  19: 10,
  23: 7,
  24: 10,
  25: 9,
  26: 10,
  27: 10,
  28: 9,
  29: 9,
  30: 9,
  31: 10,
  32: 2,
  34: 2,
  35: 4,
  36: 3,
  37: 3,
  38: 3},
 2016: {1: 10,
  2: 10,
  3: 10,
  4: 10,
  5: 10,
  6: 10,
  7: 10,
  8: 10,
  9: 10,
  10: 10,
  11: 10,
  12: 10,
  13: 10,
  14: 10,
  15: 10,
  16: 10,
  17: 10,
  18: 10,
  19: 10,
  20: 10,
  21: 10,
  22: 10,
  23: 10,
  24: 10,
  25: 10,
  26: 10,
  27: 10,
  28: 10,
  29: 10,
  30: 10,
  31: 10,
  32: 10,
  33: 9,
  34: 10,
  35: 10,
  36: 9,
  37: 10,
  38: 10},
 2017: {2: 1, 3: 4},
 2021: {1: 5}}

Tendo tais quantidades por rodada como faltantes, uma estratégia é inferir, por proximidade, as datas cujas rodadas tenham, pelo menos, 5 jogos com datas. As demais rodadas serão inferidas tendo como base as datas das rodadas correspondentes nos outros anos, por meio de uma média. Dessa forma, dos jogos acima, 2013 e 2016 serão inferidos, inteiramente, pelos demais anos enquanto 2015 terá essa abordagem apenas até a rodada 31, pois as próximas se enquadram no caso em que temos uma boa quantidade de dados, logo as datas podem ser inferidas por proximidade.

In [13]:
df['New_Date'] = df['Date']
i = 0
while df.loc[i, 'Competition'] == 'Serie A':
    i_init = i
    rd = df.loc[i, 'Round']
    nan_count = 0
    dates = []
    while df.loc[i, 'Round'] == rd:
        if type(df.loc[i, 'New_Date']) != str:
            nan_count += 1
        else:
            dates.append(df.loc[i, 'New_Date'])
            
        i += 1
        
    if nan_count <= 5:
        i = i_init
        while df.loc[i, 'Round'] == rd:
            if type(df.loc[i, 'New_Date']) != str:
                df.loc[i, 'New_Date'] = choice(dates)
                
            i += 1

Feito isso, podemos ver o resultado para as rodadas 34 e 35 de 2015:

In [14]:
df.loc[1090:1109]

,Competition,Team 1,Team 2,Result,Round,Year,Date,Date_by_Nearest,New_Date
1090,Serie A,Internacional / RS,Ponte Preta / SP,1 x 0,34,2015,07/11/2015,07/11/2015,07/11/2015
1091,Serie A,Corinthians / SP,Coritiba / PR,2 x 1,34,2015,07/11/2015,07/11/2015,07/11/2015
1092,Serie A,Athlético / PR,Avaí / SC,2 x 1,34,2015,07/11/2015,07/11/2015,07/11/2015
1093,Serie A,Fluminense / RJ,Chapecoense / SC,2 x 3,34,2015,07/11/2015,07/11/2015,07/11/2015
1094,Serie A,Palmeiras / SP,Vasco da Gama / RJ,0 x 2,34,2015,08/11/2015,08/11/2015,08/11/2015
1095,Serie A,Cruzeiro / MG,São Paulo / SP,2 x 1,34,2015,08/11/2015,08/11/2015,08/11/2015
1096,Serie A,Figueirense / SC,Atlético / MG,0 x 1,34,2015,NaN,08/11/2015,07/11/2015
1097,Serie A,Flamengo / RJ,Goiás / GO,4 x 1,34,2015,NaN,08/11/2015,08/11/2015
1098,Serie A,Sport / PE,Grêmio / RS,1 x 0,34,2015,08/11/2015,08/11/2015,08/11/2015
1099,Serie A,Joinville / SC,Santos / SP,0 x 0,34,2015,08/11/2015,08/11/2015,08/11/2015


Note que as datas inferidas se aproximam das datas inferidas por proximidade.

Nosso próximo passo agora é lidar com as datas ainda não tratadas, isso é, os anos de 2013 e 2016, além de boa parte de 2015. Para isso, vamos pegar os anos que temos datas completas, com exceção dos anos de 2020 e 2021, pois seus calendários foram afetados pela COVID-19.

In [15]:
fix = []
count = {}
for i in range(3221):
    if type(df.loc[i, 'New_Date']) != str:
        year = df.loc[i, 'Year']
        rd = df.loc[i, 'Round']
        fix.append([i, year, rd])
        if year in count:
            if rd in count[year]:
                count[year][rd] += 1
            else:
                count[year][rd] = 1
        
        else:
            count[year] = {}
            count[year][rd] = 1

print('Ainda existem', len(fix), 'jogos sem data')

years = [*range(2013, 2022)]
i = 0
while i < len(years):        
    if years[i] in count:
        years.remove(years[i])
    else:
        i += 1
        
if 2020 in years: years.remove(2020) # calendário afetado pela COVID-19
if 2021 in years: years.remove(2021) # calendário afetado pela COVID-19
years

Ainda existem 893 jogos sem data


[2014, 2017, 2018, 2019]

Agora, vamos calcular o calendário médio desses 4 anos para ver em quais datas, em média, ocorreu cada rodada.

In [16]:
calendario = {}
for i in range(3221):
    if df.loc[i, 'Year'] in years:
        year = df.loc[i, 'Year']
        rd = df.loc[i, 'Round']
        if rd not in calendario:
            calendario[df.loc[i, 'Round']] = {}
        
        if year not in calendario[rd]:
            calendario[rd][year] = [date2int(df.loc[i, 'New_Date'])]
        else:
            calendario[rd][year].append(date2int(df.loc[i, 'New_Date']))
            
for rd in calendario:
    for year in calendario[rd]:
        calendario[rd][year] = np.array(calendario[rd][year]).mean()
        
    a = round(np.array(list(calendario[rd].values())).mean())
    calendario[rd] = date.fromordinal(a)
    
for i in range(1, 10):
    # para não ter uma grande pausa em junho, o que não é normal, mas aconteceu por
    # causa da Copa do Mundo, em 2014
    calendario[i] += timedelta(days = 12)
    
calendario

{1: datetime.date(2017, 5, 8),
 2: datetime.date(2017, 5, 14),
 3: datetime.date(2017, 5, 24),
 4: datetime.date(2017, 5, 28),
 5: datetime.date(2017, 6, 3),
 6: datetime.date(2017, 6, 8),
 7: datetime.date(2017, 6, 15),
 8: datetime.date(2017, 6, 17),
 9: datetime.date(2017, 6, 24),
 10: datetime.date(2017, 7, 1),
 11: datetime.date(2017, 7, 5),
 12: datetime.date(2017, 7, 11),
 13: datetime.date(2017, 7, 25),
 14: datetime.date(2017, 7, 30),
 15: datetime.date(2017, 8, 6),
 16: datetime.date(2017, 8, 9),
 17: datetime.date(2017, 8, 15),
 18: datetime.date(2017, 8, 21),
 19: datetime.date(2017, 8, 27),
 20: datetime.date(2017, 9, 1),
 21: datetime.date(2017, 9, 6),
 22: datetime.date(2017, 9, 11),
 23: datetime.date(2017, 9, 18),
 24: datetime.date(2017, 9, 22),
 25: datetime.date(2017, 9, 27),
 26: datetime.date(2017, 10, 3),
 27: datetime.date(2017, 10, 10),
 28: datetime.date(2017, 10, 12),
 29: datetime.date(2017, 10, 20),
 30: datetime.date(2017, 10, 24),
 31: datetime.date(2017,

Feito isso, podemos encaixar esse calendário nos anos faltantes, usando as datas que existem lá como parâmetros de ajuste, facilitando no encaixe do calendário. Para tanto, vamos calcular o dia médio que temos para as rodadas desse ano.

In [17]:
round_days_to_fit = {}
for i in range(3221):
    if df.loc[i, 'Year'] in count:
        year = df.loc[i, 'Year']
        rd = df.loc[i, 'Round']
        if type(df.loc[i, 'New_Date']) == str:
            if rd not in round_days_to_fit:
                round_days_to_fit[df.loc[i, 'Round']] = {}

            if year not in round_days_to_fit[rd]:
                round_days_to_fit[rd][year] = [date2int(df.loc[i, 'New_Date'])]
            else:
                round_days_to_fit[rd][year].append(date2int(df.loc[i, 'New_Date']))
            
for rd in round_days_to_fit:
    for year in round_days_to_fit[rd]:
        round_days_to_fit[rd][year] = round(np.array(round_days_to_fit[rd][year]).mean())
        round_days_to_fit[rd][year] = date.fromordinal(round_days_to_fit[rd][year])
    
round_days_to_fit

{1: {2013: datetime.date(2013, 5, 26)},
 2: {2013: datetime.date(2013, 6, 2)},
 3: {2013: datetime.date(2013, 6, 1)},
 4: {2013: datetime.date(2013, 6, 5)},
 7: {2013: datetime.date(2013, 7, 14)},
 8: {2013: datetime.date(2013, 7, 21)},
 13: {2013: datetime.date(2013, 8, 11)},
 15: {2013: datetime.date(2013, 8, 18)},
 17: {2013: datetime.date(2013, 9, 1)},
 19: {2013: datetime.date(2013, 9, 8)},
 23: {2013: datetime.date(2013, 9, 21), 2015: datetime.date(2015, 9, 6)},
 26: {2013: datetime.date(2013, 10, 6)},
 27: {2013: datetime.date(2013, 10, 9)},
 28: {2013: datetime.date(2013, 10, 13), 2015: datetime.date(2015, 9, 27)},
 29: {2013: datetime.date(2013, 10, 16), 2015: datetime.date(2015, 10, 4)},
 30: {2013: datetime.date(2013, 10, 20), 2015: datetime.date(2015, 10, 15)},
 31: {2013: datetime.date(2013, 10, 27)},
 32: {2013: datetime.date(2013, 11, 3), 2015: datetime.date(2015, 10, 25)},
 33: {2013: datetime.date(2013, 11, 10),
  2015: datetime.date(2015, 11, 1),
  2016: datetime.date

# Rever isso

Agora temos os dias que, em média, ocorreram os jogos dessas rodadas. Outra coisa importante a se destacar é que os jogos eram, em geral, concentrados no sábado e domingo ou então  quarta e quinta (com raros encontros nas terças-feiras), sendo que o intervalo mínimo para descanso dos atletas, na época, variava entre 66 e 72 horas. Assim, um clube que jogou no domingo não poderia jogar na terça-feira, já um clube que jogou na quinta pode jogar no domingo, mas não no sábado. Entretanto, por simplicidade, vamos considerar folgas em intervalos de 3, 4 ou 7 dias, uma vez que uma variação de poucos dias será pouco influente no modelo final.

Por fim, vale ressaltar que houve uma pausa de aproximadamente um mês em 2013, em virtude da Copa das Confederações, que ocorreram aqui no Brasil.

Dito isso, vamos calcular os calendários de 2013, 2015 e 2016:

In [18]:
rds2013 = []
for rd in round_days_to_fit:
    if 2013 in round_days_to_fit[rd]:
        rds2013.append([rd, round_days_to_fit[rd][2013]])

rds2013.sort(key = lambda x : x[0])
k = 0
while len(rds2013) < 38 and k < 37:
    # corrigindo rodada dupla em um fim de semana
    if rds2013[k][0] == 2:
        rds2013[k][1] -= timedelta(days = 4)
    
    if rds2013[k][0] == 3:
        rds2013[k][1] += timedelta(days = 1)
    
    if rds2013[k][0] != rds2013[k + 1][0] - 1:
        rounds_to_add = rds2013[k + 1][0] - rds2013[k][0] - 1
        time = rds2013[k + 1][1] - rds2013[k][1]
        if time / (rounds_to_add + 1) >= timedelta(days = 7):
            for i in range(rounds_to_add):
                rds2013.insert(k + i + 1, [rds2013[k][0] + i + 1, rds2013[k + i][1] + timedelta(days = 7)])
        elif time / (rounds_to_add + 1) >= timedelta(days = 3, hours = 12):
            for i in range(rounds_to_add):
                if i % 2 == 0:
                    rds2013.insert(k + i + 1, [rds2013[k][0] + i + 1, rds2013[k + i][1] + timedelta(days = 4)])
                else:
                    rds2013.insert(k + i + 1, [rds2013[k][0] + i + 1, rds2013[k + i][1] + timedelta(days = 3)])

        else:
            for i in range(rounds_to_add):
                rds2013.insert(k + i + 1, [rds2013[k][0] + i + 1, rds2013[k + i][1] + timedelta(days = 3)])

    k += 1
    
# arrumando para ter folga durante a Copa das Confederações
rds2013[4] = [5, rds2013[3][1] + timedelta(days = 3)]
rds2013[5] = [6, rds2013[4][1] + timedelta(days = 4)]
for rd in rds2013:
    print(rd)

[1, datetime.date(2013, 5, 26)]
[2, datetime.date(2013, 5, 29)]
[3, datetime.date(2013, 6, 2)]
[4, datetime.date(2013, 6, 5)]
[5, datetime.date(2013, 6, 8)]
[6, datetime.date(2013, 6, 12)]
[7, datetime.date(2013, 7, 14)]
[8, datetime.date(2013, 7, 21)]
[9, datetime.date(2013, 7, 25)]
[10, datetime.date(2013, 7, 28)]
[11, datetime.date(2013, 8, 1)]
[12, datetime.date(2013, 8, 4)]
[13, datetime.date(2013, 8, 11)]
[14, datetime.date(2013, 8, 15)]
[15, datetime.date(2013, 8, 18)]
[16, datetime.date(2013, 8, 25)]
[17, datetime.date(2013, 9, 1)]
[18, datetime.date(2013, 9, 5)]
[19, datetime.date(2013, 9, 8)]
[20, datetime.date(2013, 9, 11)]
[21, datetime.date(2013, 9, 14)]
[22, datetime.date(2013, 9, 17)]
[23, datetime.date(2013, 9, 21)]
[24, datetime.date(2013, 9, 25)]
[25, datetime.date(2013, 9, 28)]
[26, datetime.date(2013, 10, 6)]
[27, datetime.date(2013, 10, 9)]
[28, datetime.date(2013, 10, 13)]
[29, datetime.date(2013, 10, 16)]
[30, datetime.date(2013, 10, 20)]
[31, datetime.date(2013,

In [19]:
rds2015 = []
for rd in round_days_to_fit:
    if 2015 in round_days_to_fit[rd]:
        rds2015.append([rd, round_days_to_fit[rd][2015]])

rds2015.sort(key = lambda x : x[0])
k = 0
while len(rds2015) < rds2015[-1][0] - rds2015[0][0] + 1 and k < rds2015[-1][0] - rds2015[0][0]:
    if rds2015[k][0] != rds2015[k + 1][0] - 1:
        rounds_to_add = rds2015[k + 1][0] - rds2015[k][0] - 1
        time = rds2015[k + 1][1] - rds2015[k][1]
        if time / (rounds_to_add + 1) >= timedelta(days = 7):
            for i in range(rounds_to_add):
                rds2015.insert(k + i + 1, [rds2015[k][0] + i + 1, rds2015[k + i][1] + timedelta(days = 7)])
        elif time / (rounds_to_add + 1) >= timedelta(days = 3, hours = 12):
            for i in range(rounds_to_add):
                if i % 2 == 0:
                    rds2015.insert(k + i + 1, [rds2015[k][0] + i + 1, rds2015[k + i][1] + timedelta(days = 4)])
                else:
                    rds2015.insert(k + i + 1, [rds2015[k][0] + i + 1, rds2015[k + i][1] + timedelta(days = 3)])

        else:
            for i in range(rounds_to_add):
                rds2015.insert(k + i + 1, [rds2015[k][0] + i + 1, rds2015[k + i][1] + timedelta(days = 3)])

    k += 1

# para as demais datas vamos encaixar os calendário antes e depois das datas que
# temos com um intervalo de 7 dias
if rds2015[0][0] != 1:
    dia_calendario = date(year, calendario[rds2015[0][0]].month, calendario[rds2015[0][0]].day)
    dt = rds2015[0][1] - dia_calendario
    for k in range(1, rds2015[0][0]):
        dia_calendario_rd_k = date(year, calendario[k + 1].month, calendario[k + 1].day)
        dia_insert = dia_calendario_rd_k + dt + timedelta(days = -7)
        rds2015.insert(k - 1, [k, dia_insert])

for rd in rds2015:
    print(rd)

[1, datetime.date(2015, 4, 28)]
[2, datetime.date(2015, 5, 8)]
[3, datetime.date(2015, 5, 12)]
[4, datetime.date(2015, 5, 18)]
[5, datetime.date(2015, 5, 23)]
[6, datetime.date(2015, 5, 30)]
[7, datetime.date(2015, 6, 1)]
[8, datetime.date(2015, 6, 8)]
[9, datetime.date(2015, 6, 15)]
[10, datetime.date(2015, 6, 19)]
[11, datetime.date(2015, 6, 25)]
[12, datetime.date(2015, 7, 9)]
[13, datetime.date(2015, 7, 14)]
[14, datetime.date(2015, 7, 21)]
[15, datetime.date(2015, 7, 24)]
[16, datetime.date(2015, 7, 30)]
[17, datetime.date(2015, 8, 5)]
[18, datetime.date(2015, 8, 11)]
[19, datetime.date(2015, 8, 16)]
[20, datetime.date(2015, 8, 23)]
[21, datetime.date(2015, 8, 30)]
[22, datetime.date(2015, 9, 2)]
[23, datetime.date(2015, 9, 6)]
[24, datetime.date(2015, 9, 10)]
[25, datetime.date(2015, 9, 13)]
[26, datetime.date(2015, 9, 17)]
[27, datetime.date(2015, 9, 20)]
[28, datetime.date(2015, 9, 27)]
[29, datetime.date(2015, 10, 4)]
[30, datetime.date(2015, 10, 15)]
[31, datetime.date(2015, 

In [20]:
rds2016 = []
for rd in round_days_to_fit:
    if 2016 in round_days_to_fit[rd]:
        rds2016.append([rd, round_days_to_fit[rd][2016]])

rds2016.sort(key = lambda x : x[0])
k = 0
while len(rds2016) < rds2016[-1][0] - rds2016[0][0] + 1 and k < rds2016[-1][0] - rds2016[0][0]:
    if rds2016[k][0] != rds2016[k + 1][0] - 1:
        rounds_to_add = rds2016[k + 1][0] - rds2016[k][0] - 1
        time = rds2016[k + 1][1] - rds2016[k][1]
        if time / (rounds_to_add + 1) >= timedelta(days = 7):
            for i in range(rounds_to_add):
                rds2016.insert(k + i + 1, [rds2016[k][0] + i + 1, rds2016[k + i][1] + timedelta(days = 7)])
        elif time / (rounds_to_add + 1) >= timedelta(days = 3, hours = 12):
            for i in range(rounds_to_add):
                if i % 2 == 0:
                    rds2016.insert(k + i + 1, [rds2016[k][0] + i + 1, rds2016[k + i][1] + timedelta(days = 4)])
                else:
                    rds2016.insert(k + i + 1, [rds2016[k][0] + i + 1, rds2016[k + i][1] + timedelta(days = 3)])

        else:
            for i in range(rounds_to_add):
                rds2016.insert(k + i + 1, [rds2016[k][0] + i + 1, rds2016[k + i][1] + timedelta(days = 3)])

    k += 1

# para as demais datas vamos encaixar os calendário antes e depois das datas que
# temos com um intervalo de 7 dias
if rds2016[0][0] != 1:
    dia_calendario = date(year, calendario[rds2016[0][0]].month, calendario[rds2016[0][0]].day)
    dt = rds2016[0][1] - dia_calendario
    for k in range(1, rds2016[0][0]):
        dia_calendario_rd_k = date(year, calendario[k + 1].month, calendario[k + 1].day)
        dia_insert = dia_calendario_rd_k + dt + timedelta(days = -7)
        rds2016.insert(k - 1, [k, dia_insert])
        
if rds2016[-1][0] != 38:
    dia_calendario = date(year, calendario[rds2016[-1][0]].month, calendario[rds2016[-1][0]].day)
    dt = rds2016[-1][1] - dia_calendario
    for k in range(rds2016[-1][0] + 1, 39):
        dia_calendario_rd_k = date(year, calendario[k].month, calendario[k].day)
        dia_insert = dia_calendario_rd_k + dt
        rds2016.append([k, dia_insert])

for rd in rds2016:
    print(rd)

[1, datetime.date(2016, 4, 20)]
[2, datetime.date(2016, 4, 30)]
[3, datetime.date(2016, 5, 4)]
[4, datetime.date(2016, 5, 10)]
[5, datetime.date(2016, 5, 15)]
[6, datetime.date(2016, 5, 22)]
[7, datetime.date(2016, 5, 24)]
[8, datetime.date(2016, 5, 31)]
[9, datetime.date(2016, 6, 7)]
[10, datetime.date(2016, 6, 11)]
[11, datetime.date(2016, 6, 17)]
[12, datetime.date(2016, 7, 1)]
[13, datetime.date(2016, 7, 6)]
[14, datetime.date(2016, 7, 13)]
[15, datetime.date(2016, 7, 16)]
[16, datetime.date(2016, 7, 22)]
[17, datetime.date(2016, 7, 28)]
[18, datetime.date(2016, 8, 3)]
[19, datetime.date(2016, 8, 8)]
[20, datetime.date(2016, 8, 13)]
[21, datetime.date(2016, 8, 18)]
[22, datetime.date(2016, 8, 25)]
[23, datetime.date(2016, 8, 29)]
[24, datetime.date(2016, 9, 3)]
[25, datetime.date(2016, 9, 9)]
[26, datetime.date(2016, 9, 16)]
[27, datetime.date(2016, 9, 18)]
[28, datetime.date(2016, 9, 26)]
[29, datetime.date(2016, 9, 30)]
[30, datetime.date(2016, 10, 5)]
[31, datetime.date(2016, 10

Tendo calculado tais calendários, podemos alterar essas datas no dataframe:

In [21]:
i = 0
while df.loc[i, 'Competition'] == 'Serie A':
    years = [2013, 2015, 2016]
    rds = [rds2013, rds2015, rds2016]
    if df.loc[i, 'Year'] in years:
        year = df.loc[i, 'Year']
        rdsyear = rds[years.index(year)]
        while df.loc[i, 'Year'] == year:
            rd = df.loc[i, 'Round']
            if type(df.loc[i, 'New_Date']) != str:
                d = rdsyear[rd - 1][1]
                df.loc[i, 'New_Date'] = str(d.day).zfill(2) + '/' + str(d.month).zfill(2) + '/' + str(year)
            
            i += 1
    else:
        i += 1
        
df

,Competition,Team 1,Team 2,Result,Round,Year,Date,Date_by_Nearest,New_Date
0,Serie A,Vasco da Gama / RJ,Portuguesa / SP,1 x 0,1,2013,25/05/2013,25/05/2013,25/05/2013
1,Serie A,Fluminense / RJ,Athlético / PR,2 x 1,1,2013,NaN,25/05/2013,26/05/2013
2,Serie A,Corinthians / SP,Botafogo / RJ,1 x 1,1,2013,NaN,25/05/2013,26/05/2013
3,Serie A,Vitória / BA,Internacional / RS,2 x 2,1,2013,NaN,25/05/2013,26/05/2013
4,Serie A,Grêmio / RS,Náutico / PE,2 x 0,1,2013,NaN,25/05/2013,26/05/2013
...,...,...,...,...,...,...,...,...,...
4446,Copa do Brasil,Bahia / BA,Atlético / MG,2 x 1,11,2021,04/08/2021,04/08/2021,04/08/2021
4447,Copa do Brasil,Athlético / PR,Santos / SP,1 x 0,11,2021,25/08/2021,25/08/2021,25/08/2021
4448,Copa do Brasil,Grêmio / RS,Flamengo / RJ,0 x 4,11,2021,25/08/2021,25/08/2021,25/08/2021
4449,Copa do Brasil,São Paulo / SP,Fortaleza / CE,2 x 2,12,2021,25/08/2021,25/08/2021,25/08/2021


Vejamos, para ilustrar, como ficou o início do ano de 2013:

In [22]:
df.loc[:29]

,Competition,Team 1,Team 2,Result,Round,Year,Date,Date_by_Nearest,New_Date
0,Serie A,Vasco da Gama / RJ,Portuguesa / SP,1 x 0,1,2013,25/05/2013,25/05/2013,25/05/2013
1,Serie A,Fluminense / RJ,Athlético / PR,2 x 1,1,2013,NaN,25/05/2013,26/05/2013
2,Serie A,Corinthians / SP,Botafogo / RJ,1 x 1,1,2013,NaN,25/05/2013,26/05/2013
3,Serie A,Vitória / BA,Internacional / RS,2 x 2,1,2013,NaN,25/05/2013,26/05/2013
4,Serie A,Grêmio / RS,Náutico / PE,2 x 0,1,2013,NaN,25/05/2013,26/05/2013
5,Serie A,Ponte Preta / SP,São Paulo / SP,0 x 2,1,2013,NaN,26/05/2013,26/05/2013
6,Serie A,Criciúma / SC,Bahia / BA,3 x 1,1,2013,NaN,26/05/2013,26/05/2013
7,Serie A,Cruzeiro / MG,Goiás / GO,5 x 0,1,2013,NaN,26/05/2013,26/05/2013
8,Serie A,Coritiba / PR,Atlético / MG,2 x 1,1,2013,NaN,26/05/2013,26/05/2013
9,Serie A,Santos / SP,Flamengo / RJ,0 x 0,1,2013,26/05/2013,26/05/2013,26/05/2013


# Rever daqui para baixo

Feito esse processo para a Série A, precisamos ver os jogos faltantes das Séries B e C, além da Copa do Brasil.

In [23]:
df.loc[3221:4031] # serie B

,Competition,Team 1,Team 2,Result,Round,Year,Date,Date_by_Nearest,New_Date
3221,Serie B,Palmeiras / SP,Atlético / GO,1 x 0,1,2013,25/05/2013,25/05/2013,25/05/2013
3222,Serie B,América / MG,Figueirense / SC,2 x 4,2,2013,28/05/2013,28/05/2013,28/05/2013
3223,Serie B,Atlético / GO,Joinville / SC,2 x 1,2,2013,NaN,28/05/2013,NaN
3224,Serie B,Figueirense / SC,Sport / PE,3 x 2,3,2013,NaN,01/06/2013,NaN
3225,Serie B,Palmeiras / SP,América / MG,0 x 1,3,2013,01/06/2013,01/06/2013,01/06/2013
...,...,...,...,...,...,...,...,...,...
4027,Serie B,Avaí / SC,Coritiba / PR,1 x 2,20,2021,20/08/2021,20/08/2021,20/08/2021
4028,Serie B,Vasco da Gama / RJ,Ponte Preta / SP,2 x 0,21,2021,29/08/2021,29/08/2021,29/08/2021
4029,Serie B,Náutico / PE,Vitória / BA,1 x 1,21,2021,29/08/2021,29/08/2021,29/08/2021
4030,Serie B,Coritiba / PR,Botafogo / RJ,0 x 1,21,2021,27/08/2021,27/08/2021,27/08/2021


In [24]:
calculate_missing_date(df, (3221, 4032), 'New_Date')

({'all': [406, 811],
  2013: [35, 87],
  2014: [106, 108],
  2015: [89, 89],
  2016: [90, 90],
  2017: [83, 90],
  2018: [1, 90],
  2019: [1, 74],
  2020: [1, 132],
  2021: [0, 51]},
 0.5006165228113441)

In [25]:
cleaning_data(competitions1 = ['Data/Serie B'], output_file = 'sb.csv')
df_sb = pd.read_csv('sb.csv')
calculate_missing_date(df_sb, (0, len(df_sb)), 'Date')

({'all': [1594, 3186],
  2013: [106, 312],
  2014: [372, 378],
  2015: [376, 377],
  2016: [380, 380],
  2017: [350, 379],
  2018: [3, 380],
  2019: [3, 380],
  2020: [3, 380],
  2021: [1, 220]},
 0.5003138731952291)

In [26]:
df.loc[4032:4070] # serie C

,Competition,Team 1,Team 2,Result,Round,Year,Date,Date_by_Nearest,New_Date
4032,Serie C,Santa Cruz / PE,Cuiabá / MT,1 x 0,3,2013,NaN,06/09/2021,NaN
4033,Serie C,Santa Cruz / PE,Fortaleza / CE,2 x 1,4,2013,13/07/2013,13/07/2013,13/07/2013
4034,Serie C,Fortaleza / CE,Cuiabá / MT,3 x 0,8,2013,07/08/2013,07/08/2013,07/08/2013
4035,Serie C,Cuiabá / MT,Santa Cruz / PE,1 x 3,12,2013,01/09/2013,01/09/2013,01/09/2013
4036,Serie C,Fortaleza / CE,Santa Cruz / PE,2 x 0,13,2013,08/09/2013,08/09/2013,08/09/2013
4037,Serie C,Cuiabá / MT,Fortaleza / CE,1 x 1,17,2013,06/10/2013,06/10/2013,06/10/2013
4038,Serie C,Fortaleza / CE,Cuiabá / MT,2 x 1,2,2014,NaN,06/10/2013,NaN
4039,Serie C,Cuiabá / MT,Fortaleza / CE,1 x 1,11,2014,NaN,06/10/2013,NaN
4040,Serie C,Cuiabá / MT,Fortaleza / CE,1 x 3,3,2015,NaN,06/10/2013,NaN
4041,Serie C,Juventude / RS,Portuguesa / SP,3 x 2,6,2015,NaN,06/10/2013,NaN


In [27]:
calculate_missing_date(df, (4032, 4071), 'New_Date')

({'all': [18, 39],
  2013: [1, 6],
  2014: [2, 2],
  2015: [4, 4],
  2016: [5, 6],
  2017: [6, 8],
  2018: [0, 4],
  2019: [0, 4],
  2021: [0, 5]},
 0.46153846153846156)

In [28]:
cleaning_data(competitions1 = ['Data/Serie C'], output_file = 'sc.csv')
df_sc = pd.read_csv('sc.csv')
calculate_missing_date(df_sc, (0, len(df_sc)), 'Date')

({'all': [763, 1654],
  2013: [49, 190],
  2014: [142, 142],
  2015: [191, 193],
  2016: [190, 194],
  2017: [188, 193],
  2018: [0, 194],
  2019: [0, 194],
  2020: [0, 204],
  2021: [3, 150]},
 0.46130592503022977)

In [29]:
df.loc[4071:] # CdB

,Competition,Team 1,Team 2,Result,Round,Year,Date,Date_by_Nearest,New_Date
4071,Copa do Brasil,Santa Cruz / PE,Internacional / RS,0 x 0,9,2013,NaN,04/09/2021,NaN
4072,Copa do Brasil,Joinville / SC,Santos / SP,0 x 1,10,2013,08/05/2013,08/05/2013,08/05/2013
4073,Copa do Brasil,Internacional / RS,Santa Cruz / PE,2 x 0,11,2013,NaN,08/05/2013,NaN
4074,Copa do Brasil,Avaí / SC,América / MG,0 x 3,11,2013,16/05/2013,16/05/2013,16/05/2013
4075,Copa do Brasil,Santos / SP,Joinville / SC,0 x 0,12,2013,22/05/2013,22/05/2013,22/05/2013
...,...,...,...,...,...,...,...,...,...
4446,Copa do Brasil,Bahia / BA,Atlético / MG,2 x 1,11,2021,04/08/2021,04/08/2021,04/08/2021
4447,Copa do Brasil,Athlético / PR,Santos / SP,1 x 0,11,2021,25/08/2021,25/08/2021,25/08/2021
4448,Copa do Brasil,Grêmio / RS,Flamengo / RJ,0 x 4,11,2021,25/08/2021,25/08/2021,25/08/2021
4449,Copa do Brasil,São Paulo / SP,Fortaleza / CE,2 x 2,12,2021,25/08/2021,25/08/2021,25/08/2021


Vamos calcular quantas datas estão faltando nos dados que coletamos:

In [30]:
calculate_missing_date(df, (4071, 4451), 'New_Date')

({'all': [145, 380],
  2013: [13, 35],
  2014: [23, 33],
  2015: [27, 46],
  2016: [38, 50],
  2017: [42, 47],
  2018: [0, 48],
  2019: [1, 45],
  2020: [0, 46],
  2021: [1, 30]},
 0.3815789473684211)

Podemos que pouco mais de 38% dos jogos estão sem data. Seguindo uma ideia similar a que realizamos para inferir as datas da Série A poderíamos chegar nos dados aproximados, entretanto aqui existem dois complicadores:

+ as rodadas não são concisas, isso é, como é uma competição de mata-mata, os dados da coluna Round são incorretos e

+ poderíamos tentar um calendário médio, o problema é que dos quatro anos que temos praticamente todos os dados, dois tiveram seu calendário alterado por influência da COVID-19, então não seria uma aproximação boa.

Podemos tentar analisar a competição como um todo, ao invés de analisar apenas os jogos de nosso interesse. Assim, vamos pegar todos os jogos da Copa do Brasil e ver quantos jogos estão sem data:

In [31]:
cleaning_data(competitions1 = ['Data/Copa do Brasil'], output_file = 'cdb.csv')
df_cdb = pd.read_csv('cdb.csv')
calculate_missing_date(df_cdb, (0, len(df_cdb)), 'Date')

UnicodeDecodeError at Data/Copa do Brasil/2016/Game 122.csv
No score found at Data/Copa do Brasil/2017/Game 018.csv, returned 0 x 0


({'all': [646, 1184],
  2013: [85, 137],
  2014: [137, 151],
  2015: [136, 156],
  2016: [148, 161],
  2017: [108, 113],
  2018: [27, 120],
  2019: [3, 120],
  2020: [0, 120],
  2021: [2, 106]},
 0.5456081081081081)

Notamos que, com os dados completos, a proporção de jogos sem datas aumentou consideravelmente, além de continuar com os problemas citados anteriormente. Dessa forma, uma das melhores opções seria descartar esses dados, pois não tem muito o que fazer com eles.

Apesar de estarmos descartando os dados, isso é algo que naturalmente seria analisado mais adiante, uma vez que a dinâmica da Copa do Brasil é completamente diferente da dinâmica do Brasileirão. Na Copa do Brasil há diversos casos de times pequenos eliminando clubes grandes e com tradição, uma vez que a competição é mata-mata em uma ou duas partidas (dependendo da fase). Assim, um clube pequeno que fica jogando no contra-ataque pode [derrotar (e até eliminar) um clube com mais tradição](https://www.uol.com.br/esporte/ultimas-noticias/2021/06/02/relembre-as-zebras-que-ja-derrubaram-times-grandes-na-copa-do-brasil.htm). Além disso, muitas vezes a dinâmica da partida em si muda, com os times jogando de modo mais conservador.

# colocar isso?

Por fim, uma forma de analisar a coerência dos dados se dá pela análise da classificação final. Isso é, se a classificação final dos jogos com os dados coletados coincide com a classificação final do campeonato, os dados devem estar corretos (a probabilidade de ocorrerem dois ou mais erros, um anulando o outro, é muito baixa).

Para isso, tomamos a classificação final do campeonato:

In [32]:
year = 6 # 2013 = 0, 2014 = 1, etc
br = df.loc[380 * year:379 + 380 * year, ['Team 1', 'Team 2', 'Result']]
table = classification(br)
table

,points,games,wins,draws,defeats,goals for,goals against,goal difference
Flamengo / RJ,90,38,28,6,4,86,37,49
Santos / SP,74,38,22,8,8,60,33,27
Palmeiras / SP,74,38,21,11,6,61,32,29
Grêmio / RS,65,38,19,8,11,64,39,25
Athlético / PR,64,38,18,10,10,51,32,19
São Paulo / SP,63,38,17,12,9,39,30,9
Internacional / RS,57,38,16,9,13,44,39,5
Corinthians / SP,56,38,14,14,10,42,34,8
Fortaleza / CE,53,38,15,8,15,50,49,1
Goiás / GO,52,38,15,7,16,46,64,-18


Histograma de gols por partidas???

# A fazer

### Transformação dos Dados

escrever...

### Integração dos Dados

escrever...

### Formatação dos Dados

escrever...

## IV. Modeling

+ Select modeling techniques: 
+ Generate test design: 
+ Build model: 
+ Assess model:  

## V. Evaluation                    

+ Evaluate results: 
+ Review process: 
+ Determine next steps: 

## VI. Deployment

+ Plan deployment: 
+ Plan monitoring and maintenance: 
+ Produce final report: 
+ Review project: 